In [79]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Annotated
import operator
from langgraph.types import CachePolicy
from langgraph.cache.memory import InMemoryCache
from datetime import datetime
from typing import Literal

In [80]:
def update_function(old,new):
    return old + new

class State(TypedDict):
    #messages: Annotated[list[str], update_function]
    messages: Annotated[list[str], operator.add]


graph_builder = StateGraph(
    State
    )

def node_one(state: State):

   return {
    "messages": ["Hello, nice to meet you!"]
   }

def node_two(state: State):

    return {
        "time": f"{ datetime.now()}"
       
    }

def node_three(state: State):

    return {
       
    }

def node_four(state: State):

    return {
    
    }



In [ ]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two, cache_policy=CachePolicy(ttl=20))
graph_builder.add_node("node_three", node_three)
graph_builder.add_node("node_four", node_four)

#def decide_path(state: State) -> Literal["node_three", "node_four"]:
#    if state["seed"] % 2 == 0:
#        return "node_three"
#    else:
#        return "node_four"

def decide_path(state: State):
    return state["seed"] % 2 == 0

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_conditional_edges("node_two", decide_path, {True: "node_three", False: "node_four"})
graph_builder.add_edge("node_four", END)

graph = graph_builder.compile()
graph.get_graph().draw_ascii()

print(graph.get_graph().draw_ascii())

            +-----------+               
            | __start__ |               
            +-----------+               
                   *                    
                   *                    
                   *                    
             +----------+               
             | node_one |               
             +----------+               
                   *                    
                   *                    
                   *                    
             +----------+               
             | node_two |               
             +----------+               
            ...         ...             
           .               .            
         ..                 ..          
+-----------+           +------------+  
| node_four |           | node_three |  
+-----------+           +------------+  
            ***         ***             
               *       *                
                **   **                 
             +--

In [69]:
import time 
graph = graph_builder.compile(cache=InMemoryCache())

print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)



{'messages': ['Hello, nice to meet you!']}
{'messages': ['Hello, nice to meet you!']}
{'messages': ['Hello, nice to meet you!']}
{'messages': ['Hello, nice to meet you!']}
